# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so don't need to load anything from disk.

In [1]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen because it's
# the number of letters in English alphabet.
N = 26

def idx_to_onehot(x, k=N+1):
  ones = torch.sparse.torch.eye(k).to(x.device)
  """ Converts the generated integers to one-hot vectors """
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])

class EchoDataset(torch.utils.data.IterableDataset):
  def __init__(self, delay=3, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY        = 4
DATASET_SIZE = 200000
ds           = EchoDataset(delay=DELAY, size=DATASET_SIZE)

In [2]:
input = torch.tensor([1, 2, 4, 5, 8])
print(idx_to_onehot(input))

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]])


## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`.

In [13]:
char_to_int = lambda c: 0 if c == ' ' else ord(c) - ord('a') + 1
int_to_char = lambda i: ' ' if i == 0 else chr(i + ord('a') - 1)

In [14]:
class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size):
    super().__init__()
    #TODO: initialize your submodules
    self.hidden_size = hidden_size
    self.model = torch.nn.GRU(input_size=N+1, hidden_size=hidden_size, num_layers=1, batch_first=True)
    self.decoder = torch.nn.Linear(self.hidden_size, N + 1)

  def forward(self, x):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass
    h0 = torch.zeros(1, x.shape[0], self.hidden_size)
    y, _ = self.model(x, h0)
    y = self.decoder(y)
    return y

  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z.
    # You need to map those characters to one-hot encodings of shape (len(s), N + 1),
    # then feed the on-hot encodings into your network,
    # then get the output from your network,
    # then convert the output back to a string of the same length, with 0 mapped to ' ',
    # and 1-26 mapped to a-z.

    # TODO
    encoded = list(map(char_to_int, s))
    encoded = idx_to_onehot(torch.tensor(encoded))

    result = self(encoded.unsqueeze(0))
    result = result.max(dim=-1).indices[0]
    result = list(map(int_to_char, result))

    return ''.join(result)
    
    

## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [15]:
def test_model(model):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    for c1, c2 in zip(s[:-DELAY], result[DELAY:]):
      correct += int(c1 == c2)
    total += len(s) - DELAY

  return correct / total

In [ ]:
import time
from tqdm.notebook import tqdm
from torch import nn
start_time = time.time()

HIDDEN_SIZE = 128 
BATCH_SIZE  = 64

def train_model(model, dataloader):
  # TODO: implement training code
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-4 * BATCH_SIZE)
  criterion = torch.nn.CrossEntropyLoss()
  model.train()

  if torch.cuda.is_available():
      model = model.cuda()
      criterion = criterion.cuda()
      device = torch.device("cuda:0")
  else:
      device = torch.device("cpu")

  dataloader_progress_wrapper = tqdm(dataloader)
  for data in dataloader_progress_wrapper:
    inp, label = data[0], data[1]
    x = idx_to_onehot(inp).to(device)
    y = label.to(device)

    out = model(x)
    loss = criterion(out.view(-1, N+1), label.view(-1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    dataloader_progress_wrapper.set_postfix({"loss": loss.item()})
      
model = GRUMemory(HIDDEN_SIZE)
dataloader = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE)
train_model(model, dataloader)

end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
print(accuracy)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

  0%|          | 0/3125 [00:00<?, ?it/s]

1.0
tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay stays constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [17]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):
  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

In [18]:
d = VariableDelayEchoDataset()
i = 0
for data in d:
  print(data)
  i += 1
  if i == 5: break

(tensor([16,  9, 24, 23,  6,  3,  9, 19, 25, 18,  2, 16, 20, 15, 15, 25,  6, 26,
        26, 26]), 6, tensor([ 0,  0,  0,  0,  0,  0, 16,  9, 24, 23,  6,  3,  9, 19, 25, 18,  2, 16,
        20, 15]))
(tensor([10, 19,  2, 19, 24,  7,  4,  7, 10,  6, 19, 21, 11,  2, 22, 23, 18,  6,
        13,  2]), 4, tensor([ 0,  0,  0,  0, 10, 19,  2, 19, 24,  7,  4,  7, 10,  6, 19, 21, 11,  2,
        22, 23]))
(tensor([20, 16, 15, 21, 16,  5,  7, 19,  9,  6, 18, 20, 23, 15,  8,  8,  2, 23,
        20, 10]), 6, tensor([ 0,  0,  0,  0,  0,  0, 20, 16, 15, 21, 16,  5,  7, 19,  9,  6, 18, 20,
        23, 15]))
(tensor([20, 19, 13, 10,  9,  5, 22, 21,  7, 19, 20, 12, 19,  4, 17, 21, 11,  5,
        12, 25]), 6, tensor([ 0,  0,  0,  0,  0,  0, 20, 19, 13, 10,  9,  5, 22, 21,  7, 19, 20, 12,
        19,  4]))
(tensor([17, 14, 22, 16,  7, 20,  5, 22, 12,  2, 11,  8,  7, 22, 13,  8, 19, 26,
        18, 20]), 4, tensor([ 0,  0,  0,  0, 17, 14, 22, 16,  7, 20,  5, 22, 12,  2, 11,  8,  7, 22,
        13,  8]))


### Model

And the model.

In [29]:
class VariableDelayGRUMemory(torch.nn.Module):
  def __init__(self, hidden_size, max_delay):
    super().__init__()
    #TODO
    self.hidden_size = hidden_size
    self.max_delay = max_delay
    self.model = torch.nn.GRU(
        input_size=N + 1, 
        hidden_size=hidden_size, 
        num_layers=1,
        batch_first=True)
    self.decoder = torch.nn.Linear(self.hidden_size, (self.max_delay + 1) * (N + 1))

  def forward(self, x, delays):
    # inputs:
    # x - tensor of shape (batch size, seq length, N + 1)
    # delays - tensor of shape (batch size)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)

    # TODO
    h0 = torch.zeros(1, x.shape[0], self.hidden_size)
    y, _ = self.model(x, h0)
    y = self.decoder(y)
    y = y.view(*y.shape[:-1], self.max_delay + 1, N + 1)
    delays = idx_to_onehot(delays, self.max_delay + 1)
    delays = delays.unsqueeze(1).repeat(1, x.shape[1], 1)
    y = (y * delays.unsqueeze(-1))
    y = y.sum(dim=-2)
    return y

  @torch.no_grad()
  def test_run(self, s, delay):
    # This function accepts one string s containing lowercase characters a-z,
    # and an int delay - the desired output delay.
    # You need to map those characters to one-hot encodings,
    # then feed the one-hot encodings into your network,
    # then get the output from your network,
    # then convert the output back to a string of the same length, with 0 mapped to ' ',
    # and 1-26 mapped to a-z.

    # TODO
    numbers = torch.tensor(list(map(char_to_int, s)))
    one_hots = idx_to_onehot(numbers, N+1).unsqueeze(0)
    delay = torch.tensor([delay])

    result = self(one_hots, delay)
    result = result.max(dim=-1).indices[0]

    result = list(map(int_to_char, result))

    return ''.join(result)


In [30]:
def test_variable_delay_model(model, max_delay=12):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  delay = random.randint(0, max_delay)

  total = 0
  correct = 0
  for i in range(1000):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s, delay)
    for c1, c2 in zip(s[:-delay], result[delay:]):
      correct += int(c1 == c2)
    total += len(s) - delay

  return correct / total

### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [32]:
import time
start_time = time.time()

MAX_DELAY = 12
SEQ_LENGTH = 20


from tqdm.notebook import tqdm
from torch import nn

HIDDEN_SIZE = 128
BATCH_SIZE  = 128

def train_variable_delay_model(model, dataloader):
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-4 * BATCH_SIZE)
  criterion = torch.nn.CrossEntropyLoss()

  model.train()

  if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    device = torch.device("cuda:0")
  else:
    device = torch.device("cpu")
  
  dataloader_progress_wrapper = tqdm(dataloader)
  for data in dataloader_progress_wrapper:
    inputs, delays, labels = data[0], data[1], data[2]
    inputs = idx_to_onehot(inputs).to(device)
    delays = delays.to(device)
    labels = labels.to(device)

    outputs = model(inputs, delays)
  
    loss = criterion(outputs.view(-1, N + 1), labels.view(-1))
  
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

model = VariableDelayGRUMemory(hidden_size=HIDDEN_SIZE, max_delay=MAX_DELAY)
ds = VariableDelayEchoDataset(max_delay=MAX_DELAY, size=600000, seq_length=SEQ_LENGTH)
dataloader = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE)
train_variable_delay_model(model, dataloader)

end_time = time.time()
assert end_time - start_time < 600, 'executing took longer than 10 mins'
accuracy = test_variable_delay_model(model, max_delay=MAX_DELAY)
print(accuracy)
assert accuracy > 0.95, 'accuracy is too low'
print('tests passed')

  0%|          | 0/4688 [00:00<?, ?it/s]

0.9928320550498172
tests passed
